# Recomendação um dos planos da Megaline: Smart ou Ultra.



# Contents <a id='back'></a>

* [Introdução](#intro)
* [Etapa 1. Visão geral dos dados](#data_review)
* [Etapa 2. Transformações dos dados](#tranforamation_data)
* [Etapa 3. Modelo](#model)
* [Conclusões](#end)

## Introdução <a id='intro'></a>

A operadora de celular Megaline quer desenvolver um modelo que possa analisar o comportamento do cliente e recomendar um dos planos mais recentes da Megaline: Smart ou Ultra.

Você tem acesso a dados de comportamento dos clientes que já mudaram para os novos planos. Para essa tarefa de classificação, você precisa desenvolver um modelo que escolhe o plano certo. Como você já executou a etapa de pré-processamento de dados, pode ir direto para a criação do modelo.

Desenvolva um modelo com a maior acurácia possível. Neste projeto, o limite para acurácia é 0,75. Verifique a acurácia usando o conjunto de dados de teste.

### Objetivo: 
- Como você analisou os dados após o download?
- Como você escolheu os tamanhos dos conjuntos?
- Você avaliou corretamente a qualidade dos modelos?
- Quais modelos e hiperparâmetros você utilizou?
- Quais são suas descobertas?
- Qual é a sua pontuação de acurácia?

### Etapas

- `Abra e examine o arquivo de dados. Caminho para o arquivo:`
/datasets/users_behavior.csv. Baixar o conjunto de dados

- `Divida os dados de origem em um conjunto de treinamento, um conjunto de validação e um conjunto de teste.`

- `Investigue a qualidade de diferentes modelos alterando hiperparâmetros.` 

- `Descreva brevemente os resultados do estudo.`

- `Verifique a qualidade do modelo usando o conjunto de teste.`

- `Tarefa adicional: tirar a prova real do modelo. Esses dados são maiscomplexos do que os que você está acostumado a trabalhar, então não será uma tarefa fácil. Vamos dar uma olhada mais de perto mais tarde.`

O seu projeto consistirá em cinco etapas:
 1. Visão geral dos dados
 2. Transformações dos dados
 3. Modelo

[Voltar ao Índice](#back)

## Etapa 1. Visão geral dos dados <a id='data_review'></a>

Carregue os dados das consultas, explore-os e carregue bibliotecas que você acredita que são necessárias para o projeto.

In [1]:
# Carregando todas as bibliotecas
import pandas as pd
import numpy as np

# Bibliotecas de modelos 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Bibliotecas de avaliação de modelos
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Carregue o arquivo com os dados em um DataFrame
df = pd.read_csv('C:/Users/gabri/Downloads/Corrigido/Projeto 7/users_behavior.csv')

In [3]:
# Vamos ver quantas linhas e colunas nosso conjunto de dados tem
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
# vamos exibir as primeiras 10 linhas
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


**Descrição dos dados**
- `сalls` - (número de chamadas)
- `minutes` - (duração total da chamada em minutos)
- `messages` - (número de mensagens de texto)
- `mb_used` - (Tráfego de Internet usado em MB)
- `is_ultra` - (plano para o mês atual (Ultimate - 1, Surf - 0))

[Voltar ao Índice](#back)

## Etapa 2. Transformação de dados <a id='tranforamation_data'></a>
Vamos examinar cada coluna para ver quais problemas podemos ter nelas.

In [5]:
df['calls'].describe()

count    3214.000000
mean       63.038892
std        33.236368
min         0.000000
25%        40.000000
50%        62.000000
75%        82.000000
max       244.000000
Name: calls, dtype: float64

In [6]:
df['minutes'].describe()

count    3214.000000
mean      438.208787
std       234.569872
min         0.000000
25%       274.575000
50%       430.600000
75%       571.927500
max      1632.060000
Name: minutes, dtype: float64

In [7]:
df['messages'].describe()

count    3214.000000
mean       38.281269
std        36.148326
min         0.000000
25%         9.000000
50%        30.000000
75%        57.000000
max       224.000000
Name: messages, dtype: float64

In [8]:
df['mb_used'].describe()

count     3214.000000
mean     17207.673836
std       7570.968246
min          0.000000
25%      12491.902500
50%      16943.235000
75%      21424.700000
max      49745.730000
Name: mb_used, dtype: float64

In [9]:
df['is_ultra'].value_counts(ascending=False,dropna=False)

is_ultra
0    2229
1     985
Name: count, dtype: int64

As demais colunas contêm valores corretos e estão em conformidade com as regras da boa prática de estilo.

## Etapa 3. Modelo <a id='model'></a>

### Dividindo os dados de origem em um conjunto de treinamento, um conjunto de validação e um conjunto de teste.

In [10]:
df_train_valid, df_test = train_test_split(df, test_size=0.2)
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25)

In [11]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Separamos em 3 os dados: conjunto de treinamento, um conjunto de validação e um conjunto de teste.

### Melhorando a qualidade do modelo

In [22]:
from sklearn.model_selection import GridSearchCV

In [21]:
param_grid_dtc = {
    'max_depth': range(1, 9),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy'],
    'random_state': [5432]
}

In [30]:
model_dtc = DecisionTreeClassifier()

# Criando o objeto GridSearchCV
grid_search_dtc = GridSearchCV(estimator=model_dtc, param_grid=param_grid_dtc, cv=5, scoring='accuracy')

# Executando a pesquisa em grade nos dados de treinamento
grid_search_dtc.fit(features_train, target_train)

# Obtendo os melhores parâmetros encontrados
best_params_dtc = grid_search_dtc.best_params_
print("Melhores parâmetros encontrados:", best_params_dtc)

# Obtendo a melhor pontuação de acurácia
best_accuracy_dtc = grid_search_dtc.best_score_
print("Melhor acurácia encontrada:", best_accuracy_dtc)

Melhores parâmetros encontrados: {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 5432}
Melhor acurácia encontrada: 0.787346746517731


In [31]:
param_grid_rfc = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'random_state': [5432]
}

In [32]:
# Criando o modelo RandomForestClassifier
model_rfc = RandomForestClassifier()

# Criando o objeto GridSearchCV
grid_search_rfc = GridSearchCV(estimator=model_rfc, param_grid=param_grid_rfc, cv=5, scoring='accuracy')

# Executando a pesquisa em grade nos dados de treinamento
grid_search_rfc.fit(features_train, target_train)

# Obtendo os melhores parâmetros encontrados
best_params_rfc = grid_search.best_params_
print("Melhores parâmetros encontrados:", best_params_rfc)

best_accuracy_rfc = grid_search_rfc.best_score_
print("Melhor acurácia encontrada:", best_accuracy_rfc)

Melhores parâmetros encontrados: {'C': 10, 'max_iter': 1000, 'penalty': 'l1', 'random_state': 5432, 'solver': 'liblinear'}
Melhor acurácia encontrada: 0.8065244599959627


In [33]:
param_grid_lr = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear','sag'],
    'max_iter': [1000],
    'random_state': [5432]
}

In [34]:
# Criando o modelo LogisticRegression
model_lr = LogisticRegression()

# Criando o objeto GridSearchCV
grid_search_lr = GridSearchCV(estimator=model_lr, param_grid=param_grid_lr, cv=5, scoring='accuracy')

# Executando a pesquisa em grade nos dados de treinamento
grid_search_lr.fit(features_train, target_train)

# Obtendo os melhores parâmetros encontrados
best_params = grid_search.best_params_
print("Melhores parâmetros encontrados:", best_params)

# Obtendo a melhor pontuação de acurácia
best_accuracy_lr = grid_search.best_score_
print("Melhor acurácia encontrada:", best_accuracy_lr)

Melhores parâmetros encontrados: {'C': 10, 'max_iter': 1000, 'penalty': 'l1', 'random_state': 5432, 'solver': 'liblinear'}
Melhor acurácia encontrada: 0.7432622299979813


### Aplicando a melhor algoritmo com os melhores hiperparâmetros para o modelo.

In [35]:
# Fazendo previsões nos dados de teste usando o modelo ajustado
predictions_test = grid_search_rfc.best_estimator_.predict(features_test)

# Calculando a acurácia das previsões nos dados de teste
accuracy_test = accuracy_score(target_test, predictions_test)
print("Acurácia do modelo nos dados de teste:", accuracy_test)

Acurácia do modelo nos dados de teste: 0.8180404354587869


## Conclusão geral <a id='end'></a>

Após realizar a otimização dos hiperparâmetros e avaliação dos modelos de classificação e regressão, podemos chegar a algumas conclusões:

Otimização de Hiperparâmetros: A utilização do GridSearchCV nos permitiu explorar uma variedade de combinações de hiperparâmetros para cada modelo. Isso resultou na identificação dos conjuntos de hiperparâmetros que maximizaram o desempenho dos modelos.

Com base nos resultados obtidos, recomenda-se utilizar o modelo RandomForestClassifier ajustado com os melhores hiperparâmetros para prever o plano mais adequado (Smart ou Ultra) para os clientes da Megaline com base em suas características e comportamentos.